In [17]:
import numpy as np
import pandas as pd

In [18]:
data_train=pd.read_csv("C:/Users/Dell/Desktop/cn/twitter_sentiment_analysis/training_twitter_x_y_train.csv")
data_test=pd.read_csv("C:/Users/Dell/Desktop/cn/twitter_sentiment_analysis/test_twitter_x_test.csv")

In [19]:
#labels->'negative'/'positive'/'neutral'
#train_labels is the list of labels for the training data
train_labels=[label for label in data_train['airline_sentiment']]

In [24]:
from nltk.tokenize import sent_tokenize, word_tokenize    #sentence tokenizer ,word tokenizer
train_tweets=[word_tokenize(tweet) for tweet in data_train['text']]
test_tweets=[word_tokenize(tweet) for tweet in data_test['text']]

In [22]:
#list of stopwords and punctuations
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

In [25]:
train_docs=[]
for j in range(0,len(train_tweets)):
    train_docs.append((train_tweets[j],train_labels[j]))
# now we will create our list of documents from the files
# each entry of the list is a tuple
# tuple contains 1.all words of a particular document 2.Label of that document

In [27]:
#to get the simple pos(part of speech) tag
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [28]:
#to get the pos tag for a word
from nltk import pos_tag

In [30]:
# now we are going to clean our data 
# we will remove stopwords and punctuations and lemmatize each document
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
def clean(words):
    output=[]
    for word in words:
        if word.lower() not in stops:
            pos=pos_tag(word)
            clean_word=lemmatizer.lemmatize(word,pos=get_simple_pos(pos[0][1]))
            output.append(clean_word.lower())
    return output

In [31]:
train_docs=[(clean(doc),label) for doc,label in train_docs]

In [36]:
test_docs=[ clean(doc) for doc in test_tweets]

In [40]:
#creating a list of all words that are in the documents after cleaning
all_words = []
for doc in train_docs:
    all_words += doc[0]

In [41]:
#to find the 3000 most frequent words which will act as features
import nltk
freq = nltk.FreqDist(all_words)
common = freq.most_common(3000)
features = [i[0] for i in common]

In [42]:
def get_feature_dict(words):
    current_features = {}
    words_set = set(words)
    for w in features:
        current_features[w] = w in words_set
    return current_features

In [43]:
training_data = [(get_feature_dict(doc), category) for doc, category in train_docs]
testing_data = [ get_feature_dict(doc) for doc  in test_docs]

In [44]:
from nltk import NaiveBayesClassifier as nbayes

In [45]:
clf=nbayes.train(training_data)

In [48]:
predicted_labels=[clf.classify(test) for test in testing_data]

In [49]:
np.savetxt('C:/Users/Dell/Desktop/cn/twitter_sentiment_analysis/y_pred.csv',predicted_labels, fmt = '%s')